In [ ]:
import sys, os
from pathlib import Path
#親ディレクトリのパスを取得,これはJupyter系特有の書き方
current_dir = os.path.join(Path().resolve())
# モジュールのあるパスを追加
sys.path.append(str(current_dir) + '/../')
import carats_csv_to_pickle as cctp

import pandas as pd

dates = ['20190812', '20190813', '20190814', '20190815', '20190816', '20190817', '20190818']
source_times = ['00_12', '12_18', '18_24']

# 日毎の機数を把握するためのDataFrame
number_count = pd.DataFrame(columns = ['count'])

target_airport = "RJTT"

for date in dates:
    for source_time in source_times:
        trk_frame = pd.read_csv(f'~/Desktop/201908/trk{date}_{source_time}.csv')
        trk_frame.columns = ["time", "Callsign", "Latitude", "Longitude", "Altitude", "Type"]
        ap_frame = pd.read_csv("~/programing/CARATS/MakeApt_v1/Aerodrome_utf8.txt",
                            header=None, usecols=[0, 2, 3], delim_whitespace=True, names=["ICAO", "Latitude", "Longitude"])

        ap_frame["Latitude_decimal"] =ap_frame["Latitude"].apply(lambda x: round((int(x[:2]) + int(x[2:4])/60 + int(x[4:6])/3600),5))
        ap_frame["Longitude_decimal"] =ap_frame["Longitude"].apply(lambda x: round((int(x[:3]) + int(x[3:5])/60 + int(x[5:7])/3600),5))

        filterd_trk_frame = trk_frame
        # filterd_trk_frameをCallsign, timeの順でソートする
        filterd_trk_frame = filterd_trk_frame.sort_values(by=['Callsign', 'time'], ascending=[True, True])
        first_trk = filterd_trk_frame.groupby("Callsign").first()

        first_trk[["Latitude", "Longitude", "Altitude", "time"]]


        # 各Callsignについて、最初のデータと最後のデータを取得する
        first_data = filterd_trk_frame.groupby('Callsign').first()
        last_data = filterd_trk_frame.groupby('Callsign').last()

        # last_dataが2000ft以下のものを抽出する
        landed_flights = last_data[last_data['Altitude'] <= 2000]

        # landed_flightsの緯度経度から、target_airportに着陸した航空機を抽出する。
        # 使用するデータはap_frameの緯度経度。
        # 出力するdataframeは、[Callsign, Altitude, Latitude, Longitude, Distance_to_Airport]

        #target_airportの緯度経度を取得
        df_target_airport = ap_frame[ap_frame['ICAO'] == target_airport]
        target_airport_lat = df_target_airport['Latitude_decimal'].values[0]
        target_airport_lon = df_target_airport['Longitude_decimal'].values[0]
        # landed_flightsにDistance_to_Airport列を追加
        import numpy as np
        landed_flights['Distance_to_Airport'] = np.sqrt((landed_flights['Latitude'] - target_airport_lat)**2 + (landed_flights['Longitude'] - target_airport_lon)**2) * 111.32
        # Distance_to_Airportが10km以下のものを抽出
        landed_to_target_airport = landed_flights[landed_flights['Distance_to_Airport'] <= 10.0]
        landed_to_target_airport[['Altitude', 'Latitude', 'Longitude', 'Distance_to_Airport']]

        
        if date in number_count.index:
            number_count.loc[date, 'count'] = number_count.loc[date, 'count'] + int(len(landed_to_target_airport.index))
        else:
            number_count.loc[date, 'count'] = int(len(landed_to_target_airport.index))
        

print("number_count")
print(number_count)

TypeError: string indices must be integers

In [ ]:
target_airport['Latitude_decimal'].values[0]
        target_airport_lon = target_airport['Longitude_decimal'].values[0]
        # landed_flightsにDistance_to_Airport列を追加
        import numpy as np
        landed_flights['Distance_to_Airport'] = np.sqrt((landed_flights['Latitude'] - target_airport_lat)**2 + (landed_flights['Longitude'] - target_airport_lon)**2) * 111.32
        # Distance_to_Airportが10km以下のものを抽出
        landed_to_target_airport = landed_flights[landed_flights['Distance_to_Airport'] <= 10.0]
        landed_to_target_airport[['Altitude', 'Latitude', 'Longitude', 'Distance_to_Airport']]

        
        if date in number_count.index:
            number_count.loc[date, 'count'] = number_count.loc[date, 'count'] + int(len(landed_to_target_airport.index))
        else:
            number_count.loc[date, 'count'] = int(len(landed_to_target_airport.index))
        

print("number_count")
print(number_count)